# Assignment 5
The script should not give an error. Any mistake will be consider as 0.


## Package installation

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [3]:
#!pip install selenium
#!pip install webdriver-manager

In [2]:
# this library is to manipulate browser
from selenium import webdriver

# it allows you to work with differen versions of drivers
# We call ChromeDriver
from webdriver_manager.chrome import ChromeDriverManager
import re
import time 
from selenium.webdriver.common.by import By
import pandas as pd # for table import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [3]:
pwd

'C:\\Users\\danie\\OneDrive\\Documentos\\GitHub\\Diplomado_PUCP\\Lecture_7\\Assignment_5'

## Loop final


In [7]:
#Run packages

# Create a dataframe to collect the results (out of the for loop)
results = pd.DataFrame()

#################################
# loop starts here
#################################

# Open driver

service = Service(executable_path="../chromedriver-win64/chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Enter in the website
url = 'https://infogob.jne.gob.pe/Eleccion'
driver.get(url)
time.sleep(5) #this line guarantees that the code will 'wait' in the website for 5 seconds


# First loop, to select all election rounds
for periodo in range(26, 1, -1):

    # Open first selection box **(relevant in the for loop)**
    proceso = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div')
    proceso.click()
    time.sleep(1)

    # Select "PROCESO PRESIDENCIAL"
    presidencial = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
    presidencial.click()
    time.sleep(1)

    # Open second box
    eleccion = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div')
    eleccion.click()
    time.sleep(1)
    
    # select every round
    periodo_xpath = f'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/div[2]/div[{periodo}]'
    periodo_element = driver.find_element(By.XPATH, periodo_xpath)
    periodo_element.click()

    # get text
    elecciones_name = periodo_element.text
    time.sleep(1)

    # Click in search box
    buscar = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[3]/div/button')
    buscar.click()
    time.sleep(3)

    # Click "Resultados"
    resultados = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[1]/ul/li[2]/a')
    resultados.click()
    time.sleep(3)

    # Get table
    raw_table = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]" )
    raw_table_html = raw_table.get_attribute( 'innerHTML' )

    # Organize the table
    tabla = pd.read_html(raw_table_html)[0]
    tabla_final = tabla[['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]  #keep the most important columns
    tabla_final.insert(0,'Elecciones', elecciones_name)            #insert the name of round of election in the fisrt column

    # Append the table in the results dataframe **(relevant in the for loop)**
    results = pd.concat([results, tabla_final], ignore_index=True)
    
    #Second loop to check extra pages in the html source text
    html_source = driver.page_source
    #reset number of pages counter
    pages=0

    if '<a href="javascript:void(0)">5</a>' in html_source:
        pages=5
    elif '<a href="javascript:void(0)">4</a>' in html_source:
        pages=4
    elif '<a href="javascript:void(0)">3</a>' in html_source:
        pages=3
    elif '<a href="javascript:void(0)">2</a>' in html_source:
        pages=2
    else:
        pass

    # Third loop to get the next pages if they exists
    if pages>1:
        a = range(3, pages+2, 1)
        for n in a:
            page_xpath = f'/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[4]/div[2]/ul/li[{n}]/a'
            next_page = driver.find_element(By.XPATH, page_xpath)
            next_page.click()

            # Get table
            raw_table = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]/div[2]" )
            raw_table_html = raw_table.get_attribute( 'innerHTML' )

            # Organize the table
            tabla = pd.read_html(raw_table_html)[0]
            tabla_final = tabla[['ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]  #keep the most important columns
            tabla_final.insert(0,'Elecciones', elecciones_name)            #insert the name of round of election in the fisrt column

            # Append the table in the results dataframe **(relevant in the for loop)**
            results = pd.concat([results, tabla_final], ignore_index=True)

    else:
        pass

    # We go back to search for the remaining tables
    back = driver.find_element(By.XPATH , '/html/body/div[1]/section/div[1]/a[2]')
    back.click()


ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:void(0)">...</a> is not clickable at point (1018, 492). Other element would receive the click: <a href="javascript:void(0)">...</a>
  (Session info: chrome=120.0.6099.217)
Stacktrace:
	GetHandleVerifier [0x00007FF671F42142+3514994]
	(No symbol) [0x00007FF671B60CE2]
	(No symbol) [0x00007FF671A076AA]
	(No symbol) [0x00007FF671A588F3]
	(No symbol) [0x00007FF671A568ED]
	(No symbol) [0x00007FF671A540EC]
	(No symbol) [0x00007FF671A52EA5]
	(No symbol) [0x00007FF671A46A0F]
	(No symbol) [0x00007FF671A75FEA]
	(No symbol) [0x00007FF671A463B6]
	(No symbol) [0x00007FF671A76490]
	(No symbol) [0x00007FF671A928F6]
	(No symbol) [0x00007FF671A75D93]
	(No symbol) [0x00007FF671A44BDC]
	(No symbol) [0x00007FF671A45C64]
	GetHandleVerifier [0x00007FF671F6E16B+3695259]
	GetHandleVerifier [0x00007FF671FC6737+4057191]
	GetHandleVerifier [0x00007FF671FBE4E3+4023827]
	GetHandleVerifier [0x00007FF671C904F9+689705]
	(N

In [8]:
# Quit the driver
driver.close()

#See results
results

,Elecciones,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 1931,UNION REVOLUCIONARIA,152149
1,PRESIDENCIAL 1931,PARTIDO APRISTA PERUANO,106088
2,PRESIDENCIAL 1931,PARTIDO DESCENTRALISTA,21950
3,PRESIDENCIAL 1931,PARTIDO COALICION NACIONAL,19640
4,PRESIDENCIAL 1936,PARTIDO SOCIAL DEMOCRATA,77042
...,...,...,...
132,PRESIDENCIAL 2006,PROGRESEMOS PERU,13965
133,PRESIDENCIAL 2006,PARTIDO RECONSTRUCCION DEMOCRATICA,11925
134,PRESIDENCIAL 2006,RESURGIMIENTO PERUANO,10857
135,PRESIDENCIAL 2006,Y SE LLAMA PERU,10539


In [9]:
# After finishing the loop, export the result table as an Excel file

results.to_excel('Results_group3.xlsx', index=False)